Đây là phiên bản hồi quy của Random Forest mà bạn đã học trong phân loại — và cũng là một trong những mô hình mạnh mẽ, ổn định và phổ biến nhất cho bài toán dự đoán giá trị liên tục.

Mình sẽ trình bày theo đúng phong cách quen thuộc:

🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Cải thiện Decision Tree Regressor đơn lẻ bằng cách kết hợp nhiều cây hồi quy, để:

Giảm overfitting
Tăng độ chính xác
Làm dự đoán ổn định hơn
🧠 Ý tưởng chính — “Nhiều cái đầu hơn một cái đầu!” (phiên bản hồi quy)
Mỗi cây trong rừng được huấn luyện trên:
Một tập con ngẫu nhiên có hoàn lại (bootstrap sample) từ dữ liệu gốc.
Một tập con ngẫu nhiên các đặc trưng khi chia mỗi nút.
Khi dự đoán:
→ Mỗi cây đưa ra một giá trị số
→ Kết quả cuối cùng = trung bình của tất cả các cây
💡 Ví dụ: 

Cây 1 dự đoán giá nhà = 2.1 tỷ
Cây 2 dự đoán = 2.3 tỷ
Cây 3 dự đoán = 1.9 tỷ
→ Dự đoán cuối cùng = (2.1 + 2.3 + 1.9) / 3 = 2.1 tỷ
✅ Ưu điểm:
Rất khó overfit (ngay cả khi cây rất sâu).
Xử lý tốt đặc trưng không quan trọng (tự động giảm ảnh hưởng).
Không cần chuẩn hóa dữ liệu.
Cho điểm quan trọng của đặc trưng (feature importance).
Hiệu suất cao, ít tinh chỉnh, thường tốt ngay từ đầu.
❌ Hạn chế:
Ít trực quan hơn cây đơn (vì là trung bình của nhiều cây).
Dự đoán chậm hơn một cây đơn (nhưng vẫn nhanh hơn KNN!).
Không ngoại suy tốt (giống Decision Tree).


🔹 3. Khi nào dùng Random Forest Regressor?
Bạn cần
mô hình mạnh, ít tinh chỉnh
, cho kết quả tốt ngay lập tức
Cần
giải thích chi tiết từng quyết định
(dùng Decision Tree thay vì RF)
Dữ liệu
hỗn hợp
, có nhiễu, có đặc trưng không quan trọng
Bài toán
rất lớn
(triệu mẫu) → hãy cân nhắc
XGBoost/LightGBM
(nhanh hơn)
Muốn
đánh giá đặc trưng nào quan trọng
Cần
dự đoán mượt, liên tục
(RF cho kết quả "trung bình", không mượt như hồi quy tuyến tính)
Làm
baseline mạnh
cho hầu hết bài toán hồi quy
Dữ liệu
chuỗi thời gian
có tính thứ tự → RF không tận dụng được

🎯 So sánh nhanh với Decision Tree Regressor:
Overfitting
Dễ overfit
✅
Khó overfit
Độ chính xác
Trung bình
✅
Cao hơn
Tốc độ huấn luyện
Nhanh
Chậm hơn (do nhiều cây)
Tốc độ dự đoán
Rất nhanh
Nhanh (có thể song song)
Giải thích
✅ Rất tốt
Kém (nhưng có feature importance)
Ổn định
Kém (thay đổi dữ liệu → cây khác)
✅
Ổn định



In [1]:
import numpy as np

class SimpleRandomForestRegressor:
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=2, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.trees = []

    def _bootstrap_sample(self, X, y):
        """Lấy mẫu ngẫu nhiên có hoàn lại"""
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[idxs], y[idxs]

    def fit(self, X, y):
        self.trees = []
        n_features = X.shape[1]
        if self.max_features is None:
            self.max_features = int(np.sqrt(n_features))  # hoặc dùng all cho hồi quy

        for _ in range(self.n_trees):
            # Tạo cây hồi quy mới
            tree = SimpleDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            
            # Bootstrap sample
            X_boot, y_boot = self._bootstrap_sample(X, y)
            
            # Chọn ngẫu nhiên đặc trưng (đơn giản hóa: chọn 1 lần/cây)
            feature_subset = np.random.choice(n_features, size=self.max_features, replace=False)
            X_boot_subset = X_boot[:, feature_subset]
            
            tree.fit(X_boot_subset, y_boot)
            self.trees.append((tree, feature_subset))

    def predict(self, X):
        # Dự đoán từ từng cây
        tree_preds = []
        for tree, features in self.trees:
            X_subset = X[:, features]
            preds = tree.predict(X_subset)
            tree_preds.append(preds)
        
        # Trung bình các dự đoán
        return np.mean(tree_preds, axis=0)

# --- Thử nghiệm ---
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Tạo dữ liệu
X, y = make_regression(n_samples=300, n_features=5, noise=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình tự code
rf_reg = SimpleRandomForestRegressor(n_trees=10, max_depth=6, min_samples_split=5)
rf_reg.fit(X_train, y_train)
y_pred = rf_reg.predict(X_test)

# Đánh giá
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"✅ MSE (Random Forest Regressor tự code): {mse:.2f}")
print(f"✅ R² (tự code): {r2:.4f}")

# So sánh với sklearn
from sklearn.ensemble import RandomForestRegressor
sk_rf = RandomForestRegressor(n_estimators=10, max_depth=6, min_samples_split=5, random_state=42)
sk_rf.fit(X_train, y_train)
sk_pred = sk_rf.predict(X_test)
sk_mse = mean_squared_error(y_test, sk_pred)
sk_r2 = r2_score(y_test, sk_pred)
print(f"🔍 Sklearn MSE: {sk_mse:.2f}")
print(f"🔍 Sklearn R²: {sk_r2:.4f}")

# Feature importance từ sklearn
print("\n🌟 Feature importance (từ sklearn):")
for i, imp in enumerate(sk_rf.feature_importances_):
    print(f"  Feature {i+1}: {imp:.3f}")

NameError: name 'SimpleDecisionTreeRegressor' is not defined